In [1]:
from lean_interact import Command, LeanServer, LeanREPLConfig, TempRequireProject, LocalProject
localprojectdir = "../matheye/benchmarks/"
config = LeanREPLConfig(verbose=True, project=LocalProject(localprojectdir))
print(config.__dict__)

server = LeanServer(config)


Build completed successfully.
No files to download
Decompressing 6641 file(s)
Unpacked in 66 ms
Completed successfully!
Build completed successfully.
{'lean_version': 'v4.19.0', 'project': LocalProject(directory='../matheye/benchmarks/', build=True), 'repl_git': 'https://github.com/augustepoiroux/repl', 'repl_rev': 'v1.0.8', 'cache_dir': '/home/matheye/anaconda3/lib/python3.12/site-packages/lean_interact/cache', 'memory_hard_limit_mb': None, 'verbose': True, '_stdout': None, '_stderr': None, 'repo_name': 'augustepoiroux/repl', 'cache_clean_repl_dir': '/home/matheye/anaconda3/lib/python3.12/site-packages/lean_interact/cache/augustepoiroux/repl/repl_clean_copy', '_cache_repl_dir': '/home/matheye/anaconda3/lib/python3.12/site-packages/lean_interact/cache/augustepoiroux/repl/repl_v1.0.8_v4.19.0', '_working_dir': '../matheye/benchmarks/'}


In [20]:

response = server.run(Command(cmd=
"""
import Mathlib

macro "hole_1" : tactic => `(tactic| admit)
macro "hole_2" : tactic => `(tactic| admit)
macro "hole_3" : tactic => `(tactic| admit)
macro "hole_4" : tactic => `(tactic| admit)
macro "hole_5" : tactic => `(tactic| admit)

theorem complex_have_chain (n : ℕ) (h : n > 0) : n + n ≥ n := by
  have h1 : n ≥ 1 := by
    hole_1
  have h2 : n + n ≥ n + 1 := by
    have h3 : n ≥ 1 := h1
    have h4 : n + n = n + n := rfl
    have h5 : n + 1 ≤ n + n := by
      hole_2
    hole_3
  have h6 : n + 1 ≥ n := by
    hole_4
  hole_5
""", all_tactics=True, infotree="full"))
print(response)


CommandResponse(env=4, messages=[Message(start_pos=Pos(column=8, line=10), end_pos=Pos(column=26, line=10), severity='warning', data="declaration uses 'sorry'"), Message(start_pos=Pos(column=36, line=10), end_pos=Pos(column=37, line=10), severity='warning', data='unused variable `h`\nnote: this linter can be disabled with `set_option linter.unusedVariables false`')], infotree=[{'node': {'stx': {'range': {'synthetic': False, 'start': {'line': 4, 'column': 0}, 'finish': {'line': 4, 'column': 43}}, 'pp': 'macro "hole_1" : tactic => `(tactic| admit)'}, 'elaborator': 'Lean.Elab.Command.elabMacro'}, 'kind': 'CommandInfo', 'children': [{'node': {'stx': {'range': {'synthetic': True, 'start': {'line': 4, 'column': 0}, 'finish': {'line': 4, 'column': 43}}, 'pp': 'syntax (name := tacticHole_1) (priority := 1000) "hole_1" : tactic'}, 'elaborator': 'Lean.Elab.Command.elabSyntax'}, 'kind': 'CommandInfo', 'children': [{'node': {'stx': {'range': {'synthetic': False, 'start': {'line': 4, 'column': 17},

In [23]:
response.infotree


[{'node': {'stx': {'range': {'synthetic': False,
     'start': {'line': 4, 'column': 0},
     'finish': {'line': 4, 'column': 43}},
    'pp': 'macro "hole_1" : tactic => `(tactic| admit)'},
   'elaborator': 'Lean.Elab.Command.elabMacro'},
  'kind': 'CommandInfo',
  'children': [{'node': {'stx': {'range': {'synthetic': True,
       'start': {'line': 4, 'column': 0},
       'finish': {'line': 4, 'column': 43}},
      'pp': 'syntax (name := tacticHole_1) (priority := 1000) "hole_1" : tactic'},
     'elaborator': 'Lean.Elab.Command.elabSyntax'},
    'kind': 'CommandInfo',
    'children': [{'node': {'stx': {'range': {'synthetic': False,
         'start': {'line': 4, 'column': 17},
         'finish': {'line': 4, 'column': 23}},
        'pp': 'tactic'},
       'isBinder': False,
       'expr': 'Lean.Parser.Category.tactic',
       'elaborator': None},
      'kind': 'TermInfo',
      'children': []},
     {'node': None,
      'kind': 'MacroExpansionInfo',
      'children': [{'node': {'stx': {'

In [ ]:
from lean_interact import Command, LeanServer, LeanREPLConfig, TempRequireProject, LocalProject, ProofStep
localprojectdir = "../matheye/benchmarks/"
config = LeanREPLConfig(verbose=True, project=LocalProject(localprojectdir))
print(config.__dict__)

server = LeanServer(config)
response = server.run(Command(cmd="""import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat
example (a b c d e : ℝ) (h : a = b + c) (h' : b = d - e) : a + e = d + c := by
  rw [h]
  sorry
"""))
print(response.sorries)


Build completed successfully.
No files to download
Decompressing 6641 file(s)
Unpacked in 62 ms
Completed successfully!
Build completed successfully.
{'lean_version': 'v4.19.0', 'project': LocalProject(directory='../matheye/benchmarks/', build=True), 'repl_git': 'https://github.com/augustepoiroux/repl', 'repl_rev': 'v1.0.8', 'cache_dir': '/home/matheye/anaconda3/lib/python3.12/site-packages/lean_interact/cache', 'memory_hard_limit_mb': None, 'verbose': True, '_stdout': None, '_stderr': None, 'repo_name': 'augustepoiroux/repl', 'cache_clean_repl_dir': '/home/matheye/anaconda3/lib/python3.12/site-packages/lean_interact/cache/augustepoiroux/repl/repl_clean_copy', '_cache_repl_dir': '/home/matheye/anaconda3/lib/python3.12/site-packages/lean_interact/cache/augustepoiroux/repl/repl_v1.0.8_v4.19.0', '_working_dir': '../matheye/benchmarks/'}
[Sorry(start_pos=Pos(line=9, column=2), end_pos=Pos(line=9, column=7), proof_state=0, goal="a b c d e : ℝ\nh : a = b + c\nh' : b = d - e\n⊢ b + c + e = d 

In [14]:
response = server.run(ProofStep(proof_state=0, tactic="""(
  rw [a]
  ring
)"""))
if isinstance(response, LeanError):
    print(response.message)

Lean error:
tactic 'rewrite' failed, equality or iff proof expected
  ℝ
a b c d e : ℝ
h : a = b + c
h' : b = d - e
⊢ b + c + e = d + c


In [13]:
# express response istype LeanError
from lean_interact.interface import LeanError
type(response) == LeanError
response.message

'Lean error:\n<input>:2:3: unknown tactic'

In [ ]:
response = server.run(ProofStep(proof_state=0, tactic="rw [h']"))

ProofStepResponse(proof_status='Completed', goals=[], proof_state=1)


In [21]:
from lean_interact import Command, LeanServer, LeanREPLConfig, TempRequireProject, LocalProject
localprojectdir = "../matheye/benchmarks/"
config = LeanREPLConfig(verbose=True, project=LocalProject(localprojectdir))
print(config.__dict__)

server = LeanServer(config)
response = server.run(Command(cmd="""import Mathlib
theorem eq_comm_demo (x y : ℕ) : x = y ↔ y = x := by
  constructor
  · 
    have a: x = y -> y = x := by
      intro h 
      rw [<- h]
      rw [h]
      <;>linarith
    exact a
  · 
    rw [a]
    rw [h]

""", all_tactics=True))

for tactic in response.tactics:
    print(tactic.tactic + " " + str(tactic.start_pos) + " " + str(tactic.end_pos))

constructor Pos(line=3, column=2) Pos(line=3, column=13)
have a : x = y -> y = x := by
  intro h
  rw [<- h]
  rw [h] <;> linarith Pos(line=5, column=4) Pos(line=9, column=17)
intro h Pos(line=6, column=6) Pos(line=6, column=13)
rw [<- h] Pos(line=7, column=6) Pos(line=7, column=15)
exact a Pos(line=10, column=4) Pos(line=10, column=11)
rw [a] Pos(line=12, column=4) Pos(line=12, column=10)


In [27]:
from lean_interact import ProofStep


In [28]:
response = server.run(ProofStep(proof_state=1, tactic="""(
nlinarith
)"""))
print(response)

ProofStepResponse(proof_status="Error: kernel type check failed: (kernel) declaration has metavariables '[anonymous]'", goals=[], proof_state=3)
